In [1]:
import matplotlib.pyplot as plt
import numpy as np
from markov import find_steady_state,convergence,find_stable_distribution_matrix

# 1. Introduzione

Abbiamo un grafo che ci dice quale sarà la transizione di un agente step dopo step. Ogni arco ha una probabilità. Un problema consiste nel vedere verso cosa convergerà il comportamento degli agenti.

Rappresentiamo il grafo come una matrice di adiacenza. Nella posizione (i,j) è presente la probabilità che nel prossimo step l'agente passi dal nodo i al nodo j. Associamo, per ogni nodo i definiamo il numero di agenti che si trovano in quel nodo allo step 0

$$A \cdot T_0 $$

Dove $T_0$ è un vettore colonna con i valori iniziali. Il risultato del prodotto è un vettore colonna che corrisponde a $T_1$.

Quindi, in generale vale:

$$T_{i+1} = A \cdot T_i$$

Naturalmente, il numero di agenti nel vettore $T_i$ è espresso in percentuale, come valore $\in [0,1]$. Il prossimo stato è detto next state o future state.

# 2. Markov Chain

Perché sono chiamate chains? Si basa sulle successive moltiplicazioni
Ogni volta, il vettore risultante deve essere tale che la somma degli elementi sia 1.

# 3. Metodo alternativo

Un modo alternativo è fare così:

$$T_{i} = A^{i} \cdot T_0$$


Un aspetto interessante è che il vettore sembra variare molto velocemente all'inizio per poi convergere a un vettore stabile finale.

Se la matrice cambia, il vettore dovrà convergere nuovamente a un nuovo valore. 

# 4. Stocasticità e regolarità

## 4.1 Stocasticità

Una matrice è stocastica se la somma delle righe/colonne è 1. Per usare le markov chain le matrici devono essere stocastiche. 

Se è 1 se si sommano le colonne

$$B = P \cdot A$$

Se è 1 se si sommano le righe

$$B = A \cdot P$$

Con A vettore colonna e P matrice di transizione.


## 4.2 Regolarità

Una matrice stocastica è anche regolare se $P^n$ $(n>1)$ ha solo entry > 0, quindi non negative e diverse da zero. 


# 5. Markov chain regolari
Se P è una matrice regolare ci sarà un $P^n$ dove $P^n X_0 = \hat{X}$ dove $\hat{X}$ è la stable distribution matrix. O dove $P^{n+1} = P^n$, allora $P,P^2,P^3,...$ è una markov chain regolare.

Quindi, nel caso delle markov chain regolari, per ottenere il vettore a convergenza basta moltiplicare lo stato iniziale con la matrice di transizione a convergenza (la stable distribuzion matrix).


<hr>

Nota che la matrice A è tale che, se presa verticalmente, i valori sommati sono pari a 1, mentre nella versione trasposta, ovviamente, la somma sulle righe fa 1

In [78]:
A = np.array([
    [0.8, 0.2, 0.1],
    [0.1, 0.7, 0.3],
    [0.1, 0.1, 0.6]
])
t_curr = np.array([0.4, 0.24, 0.36])


# you can do A @ t_curr.T
# or t_curr @ A.T

A @ t_curr.T

array([0.404, 0.316, 0.28 ])

In [94]:
# la funzione presume che A sia tale che la somma delle colonne sia 1
def convergence(A, t_curr, eps=1e-20):
    assert np.allclose(A.sum(axis=0), 1)
    t = t_curr.copy()
    t_next = t @ A.T
    while (abs(t_next - t)).sum() > eps:
        t = t_next
        t_next = t @ A.T
    return t_next

def get_nth_step(A, t_curr, n):
    t = t_curr.copy()
    for _ in range(n):
        t = t @ A.T
    return t

print(convergence(A, t_curr))

print(get_nth_step(A, t_curr, 4))

[0.45 0.35 0.2 ]
[0.432784 0.357216 0.21    ]


Usiamo il metodo alternativo che fa uso della potenza della matrice di transizione A

In [74]:
def get_nth_state_alternative(A, t_curr, n):
    A_n = np.linalg.matrix_power(A, n)
    return t_curr @ A_n.T

print(get_nth_state_alternative(A, t_curr, 4))

[0.432784 0.357216 0.21    ]


In [75]:
A = np.array([
    [0.92, .03, .02, .01],
    [.02, .94, .02, .01],
    [.01, .01, .90, .01],
    [0.05, .02, .06, .97]
])
# row  : to 
# col : from 

t_0 = np.array([0.40, 0.32, 0.18, 0.10])

convergence(A, t_0, eps=1e-20)


array([0.1609538 , 0.17883756, 0.09090909, 0.56929955])

In [95]:
A = np.array([
    [0.8, 0.1],
    [0.2, 0.9]
])
print(convergence(A,[1,0]))
print(convergence(A,[0.5,0.5]))
print(convergence(A,[0,1]))

[0.33333333 0.66666667]
[0.33333333 0.66666667]
[0.33333333 0.66666667]


Indipendentemente dallo stato iniziale lo stato finale è sempre lo stesso

In [174]:
def convergence_stable_distr_matrix(A,t_0,eps=1e-20):
    assert np.allclose(A.sum(axis=0), 1)
    A_cur = A.copy()
    A_next = A_cur @ A
    while (abs(A_next - A_cur)).sum() > eps:
        A_cur = A_next
        A_next = A_cur @ A

    return t_0 @ A_next.T

# è stocastica
A = np.array([
    [0.5, 0.25],
    [0.5, 0.75]
])

print("Using stable distribution matrix")
print(convergence_stable_distr_matrix(A,[.4,.6]))
print()
print("Vanilla method")
print(convergence(A,[.4,.6]))

Using stable distribution matrix
[0.33333333 0.66666667]

Vanilla method
[0.33333333 0.66666667]


I risultati sono identici!

Come facciamo a trovare la stable distribution matrix?

E' la soluzione di

A x = x

che sarebbe l'autovettore associato all'autovalore 1

In [181]:
def find_steady_state(A):
    assert np.allclose(A.sum(axis=0), 1)
    eval,evect = np.linalg.eig(A)
    index = np.argmin(np.abs(eval - 1))
    steady_state = evect[:, index].real
    steady_state /= np.sum(steady_state)
    return steady_state
def find_stable_distribution_matrix(A):
    ss = find_steady_state(A)
    return np.full_like(A,ss).T
    
A = np.array([
    [0.9, 0.5],
    [0.1, 0.5]
])

print(find_steady_state(A))
print()
print(find_stable_distribution_matrix(A))
    

[0.83333333 0.16666667]

[[0.83333333 0.83333333]
 [0.16666667 0.16666667]]


In [182]:
A = np.array([
    [0.95, 0.5],
    [0.05, 0.5]
])

find_steady_state(A)

array([0.90909091, 0.09090909])

In [185]:
A = np.array([
    [0.8,0.2,0.3],
    [0.1,0.7,0.1],
    [0.1,0.1,0.6]
])

find_steady_state(A)

array([0.55, 0.25, 0.2 ])

In [187]:
A = np.array([
    [0.9,0.4],
    [0.1,0.6]
])

find_steady_state(A)
    

array([0.8, 0.2])

Absorbing Markov Chain


In [189]:
A = np.array([
    [1, 0.2, 0.2],
    [0, 0.6, 0.1],
    [0, 0.2, 0.7]
])

print(find_steady_state(A))
print()
print(A.T)


[1. 0. 0.]

[[1.  0.  0. ]
 [0.2 0.6 0.2]
 [0.2 0.1 0.7]]


In [191]:
A = np.array([
    [1, 0, 0.2, 0.1],
    [0, 1, 0.1, 0.2],
    [0, 0, 0.5, 0.1],
    [0, 0, 0.2, 0.6]
])

find_steady_state(A)

array([1., 0., 0., 0.])

Nella matrice di transizione l'elemento $p_{ij}$ corrisponde alla probabilità che l'agente si sposti dal j-esimo stato all'i-esimo stato. Se quindi $p_{ii}=1$ la popolazione non cambierà (naturalmente $p_{i=1...n,j}=0$, ($i\neq j$)), abbiamo una absorbing markov chain.

In [16]:
A = np.array([
    [1, .2, .3],
    [0, .7, .1],
    [0, .1, .6]
])

find_steady_state(A)

[[0.2 0.3 1. ]
 [0.7 0.1 0. ]
 [0.1 0.6 0. ]]


array([0.42654028, 0.33175355, 0.24170616])

Per ottenere la stable matrix delle absorbing markov chain è necessario riportarle nella forma normale. E' necessario portare la colonna targe a sinistra.

In [25]:
A = np.array([
    [.3,0,0],
    [0,1,.6],
    [.7,0,.4]
])
'''
Dobbiamo cambiare non solo l'ordine
delle colonne ma anche quello delle
righe, i.e., l'idea è quella di 
riportarsi alla matrice identità
'''
print(find_steady_state(A))
print()
# swap row
A = A[[1,0,2],:]
# swap column
A= A[:,[1,0,2]]
print(A)
print()
find_steady_state(A)


[0. 1. 0.]

[[1.  0.  0.6]
 [0.  0.3 0. ]
 [0.  0.7 0.4]]



array([1., 0., 0.])

Una volta ottenuta nella forma normale è possibile usare un metodo particolare per il calcolo.

$$\hat{P}=P^n$$

Con n grande oppure, portando P nella forma standard. Possiamo descrivere la matrice in 4 sotto-matrici:
- in alto a sinistra la matrice identità
- in basso a sinistra solo zeri
- in alto a destra una matrice S
- in basso a destra una matrice R

La matrice steady sarà così definita:
- in alto a sinistra la matrice identità
- in basso a sinistra zeri
- in basso a destra zeri
- in alto a destra $S(I-R)^{-1}$


In [32]:
A = np.array([
    [0.5,0,.3],
    [0,1,.2],
    [0.5,0,.5]
])

# questo metodo naive non è detto funzioni sempre
print(find_stable_distribution_matrix(A))
print()
print(find_steady_state(A))

[[0. 0. 0.]
 [1. 1. 1.]
 [0. 0. 0.]]

[0. 1. 0.]


In [3]:
A = np.array([
    [1,0,.2,.3],
    [0,1,.2,.1],
    [0,0,.5,.2],
    [0,0,.1,.4]
])

# come possiamo vedere qui, il metodo
# find_steady_state non funziona, questo
# perché questa è una absorbing markov

print(find_stable_distribution_matrix(A))
print()
print(find_steady_state(A))
print()
print(convergence(A,[.25,.25,.25,.25],eps=1e-350))
print()
print(convergence(A,[.1,.5,.1,.3],eps=1e-350))

[[1. 1. 1. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

[1. 0. 0. 0.]

[0.55357143 0.44642857 0.         0.        ]

[0.35714286 0.64285714 0.         0.        ]
